In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from spacy import load as spacy_load
from transformers import pipeline
from datasets import load_dataset

print("Imported successfully")

Imported successfully


In [2]:
# Chargement des chemins des modèles
spacy_model_path = r"C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/models/spacy"
distilbert_model_path = r"C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/models/distilbert"

print("Model paths loaded successfully")

Model paths loaded successfully


In [3]:
# Chargement des datasets
test_dataset_path = r"C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/tokens/token.csv"
test_data = load_dataset('csv', data_files=test_dataset_path, delimiter=';')['train']

print("Dataset loaded successfully")

Dataset loaded successfully


In [4]:
# Chargement du modèle spaCy
print("Chargement du modèle spaCy...")
spacy_model = spacy_load(spacy_model_path)

# Chargement du modèle DistilBERT
print("Chargement du modèle DistilBERT...")
distilbert_pipeline = pipeline(
    "token-classification",
    model=distilbert_model_path,
    aggregation_strategy="simple"
)

print("Loaded models successfully")

Chargement du modèle spaCy...
Chargement du modèle DistilBERT...

Loaded models successfully


In [ ]:
# Préparation des fonctions d'évaluation
def evaluate_spacy(spacy_model, dataset):
    y_true, y_pred = [], []
    for record in dataset:
        # Convertir la chaîne de tokens en liste sans risquer une erreur de syntaxe
        tokens = record["tokens"].strip('[]').replace("'", "").split(', ')
        
        # Convertir les labels en une liste de labels
        true_labels = eval(record["ner_tags"])  # Évaluation correcte de la chaîne NER

        # Traiter le texte avec le modèle spaCy
        doc = spacy_model(" ".join(tokens))
        pred_labels = []
        
        # Récupérer les entités prédites
        for token in doc:
            pred_labels.append(token.ent_type_ if token.ent_type_ else "O")
        
        y_true.extend(true_labels)
        y_pred.extend(pred_labels)
    
    return y_true, y_pred


def evaluate_distilbert(distilbert_pipeline, dataset):
    y_true, y_pred = [], []
    for record in dataset:
        tokens = eval(record["tokens"])
        true_labels = eval(record["ner_tags"])
        sentence = " ".join(tokens)
        predictions = distilbert_pipeline(sentence)
        pred_labels = ["O"] * len(tokens)
        for entity in predictions:
            for i in range(entity["start"], entity["end"]):
                pred_labels[i] = entity["entity_group"]
        y_true.extend(true_labels)
        y_pred.extend(pred_labels)
    return y_true, y_pred

# Exécution des évaluations
print("Évaluation du modèle spaCy...")
spacy_y_true, spacy_y_pred = evaluate_spacy(spacy_model, test_data)

print("Évaluation du modèle DistilBERT...")
distilbert_y_true, distilbert_y_pred = evaluate_distilbert(distilbert_pipeline, test_data)

Évaluation du modèle spaCy...


In [ ]:
# Calcul des métriques
def compute_metrics(y_true, y_pred, labels):
    report = classification_report(y_true, y_pred, labels=labels, output_dict=True)
    f1 = f1_score(y_true, y_pred, average="weighted", labels=labels)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    return report, f1, cm

labels = ["O", "B-DEP", "I-DEP", "B-ARR", "I-ARR"]

print("Calcul des métriques pour spaCy...")
spacy_report, spacy_f1, spacy_cm = compute_metrics(spacy_y_true, spacy_y_pred, labels)

print("Calcul des métriques pour DistilBERT...")
distilbert_report, distilbert_f1, distilbert_cm = compute_metrics(distilbert_y_true, distilbert_y_pred, labels)

In [ ]:
# Visualisation des résultats
def plot_confusion_matrix(cm, labels, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.ylabel('Vrai')
    plt.xlabel('Prédit')
    plt.show()

print("Matrice de confusion pour spaCy")
plot_confusion_matrix(spacy_cm, labels, "Matrice de Confusion - spaCy")

print("Matrice de confusion pour DistilBERT")
plot_confusion_matrix(distilbert_cm, labels, "Matrice de Confusion - DistilBERT")

In [ ]:
# Comparaison des performances
comparison_df = pd.DataFrame({
    "Metric": ["Precision", "Recall", "F1-Score", "Accuracy"],
    "spaCy": [
        spacy_report["macro avg"]["precision"],
        spacy_report["macro avg"]["recall"],
        spacy_report["macro avg"]["f1-score"],
        spacy_report["accuracy"]
    ],
    "DistilBERT": [
        distilbert_report["macro avg"]["precision"],
        distilbert_report["macro avg"]["recall"],
        distilbert_report["macro avg"]["f1-score"],
        distilbert_report["accuracy"]
    ]
})
print(comparison_df)

In [ ]:
# Visualisation des comparaisons
comparison_df.set_index("Metric").plot(kind="bar", figsize=(10, 6))
plt.title("Comparaison des métriques entre spaCy et DistilBERT")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.legend(title="Modèles")
plt.show()

In [ ]:
# Conclusion scientifique
print("Résumé des performances :")
print("spaCy F1-Score : ", spacy_f1)
print("DistilBERT F1-Score : ", distilbert_f1)
if distilbert_f1 > spacy_f1:
    print("DistilBERT est globalement meilleur, particulièrement grâce à ses capacités sur des contextes complexes.")
else:
    print("spaCy est meilleur, possiblement grâce à sa rapidité et simplicité pour des tâches bien définies.")